In [29]:
# ==========================================
# CELL 1: SETUP
# ==========================================
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv google-search-results

In [39]:
import os
os.environ["SERPAPI_API_KEY"] = ""
#from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

# 🔑 SETUP API KEY
os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

print("✅ Libraries installed and LLM ready.")

✅ Libraries installed and LLM ready.


In [49]:
from langchain_community.utilities import SerpAPIWrapper

def news_agent(question: str) -> str:
    prompt = f"""
You are a News Reporter.

Expertise:
- News
- Headlines
- Text explanations

LIMITATION:
- You CANNOT handle prices, numbers, ratios, or stock values.

RULE:
If the user asks about price, trading value, numbers, ratios, or money,
respond with EXACTLY:
FAIL_NEEDS_NUMBERS

Otherwise respond with:
HANDLE_NEWS

Question:
{question}
"""

    decision = llm.invoke(prompt).content.strip()

    if decision == "FAIL_NEEDS_NUMBERS":
        return "FAIL_NEEDS_NUMBERS"

    # 🔍 Fetch latest news using SERP
    search = SerpAPIWrapper()
    result = search.results(question)

    articles = result.get("news_results", [])

    if not articles:
        return "No recent news found."

    headlines = [
        a.get("title", "No title available")
        for a in articles[:3]
    ]

    summary_prompt = f"""
Summarize the following headlines clearly and concisely:

{headlines}
"""

    return llm.invoke(summary_prompt).content.strip()

In [67]:
def stock_broker_agent(question: str) -> str:
    """
    Handles ONLY prices, numbers, ratios.
    """

    prompt = f"""
You are a Stock Broker.

Expertise:
- Stock prices
- Numbers
- Ratios

Limitation:
- Do NOT explain news or headlines.

Answer ONLY with numeric/price-related information and at leat just provide information about mumber parser the content from web.

Question:
{question}
"""
    
    print("💹 Stock Broker handling the query...\n")

    search = SerpAPIWrapper()
    data = search.run(question)
    return llm.invoke(prompt)

In [68]:
def swarm_intelligence(question: str) -> str:
    print("\n📰 Asking News Agent first...\n")

    news_response = news_agent(question)

    if "FAIL_NEEDS_NUMBERS" in news_response:
        print("❌ News Agent failed... Handing off to Broker.\n")
        broker_response = stock_broker_agent(question)
        return broker_response
    return news_response

In [69]:
question = "What is Zomato trading at today?"

answer = swarm_intelligence(question)
print("✅ FINAL ANSWER:\n", answer)


📰 Asking News Agent first...

❌ News Agent failed... Handing off to Broker.

💹 Stock Broker handling the query...

✅ FINAL ANSWER:
 content='Zomato (NSE: ZOMATO) is currently trading at ₹196.25.'
